# Sprint23課題 深層学習スクラッチリカレントニューラルネットワーク

## この課題の目的

- スクラッチを通してRNNの基礎を理解する

## スクラッチによる実装

NumPyなど最低限のライブラリのみを使いアルゴリズムを実装していきます。

Sprint11で作成したディープニューラルネットワークのクラスを拡張する形でRNNを作成します。

## 【問題1】SimpleRNNのフォワードプロパゲーション実装

SimpleRNNのクラスSimpleRNNを作成してください。基本構造はFCクラスと同じになります。

今回はバッチサイズをbatch_size、入力の特徴量数をn_features、RNNのノード数をn_nodesとして表記します。活性化関数はtanhとして進めますが、これまでのニューラルネットワーク同様にReLUなどに置き換えられます。

フォワードプロパゲーションの数式は以下のようになります。ndarrayのshapeがどうなるかを併記しています。

$$a_t = x_{t}\cdot W_{x} + h_{t-1}\cdot W_{h} + b$$

$$h_t = tanh(a_t)$$

$a_t$ : 時刻tの活性化関数を通す前の状態 (batch_size, n_nodes)

$h_t$ : 時刻tの状態・出力 (batch_size, n_nodes)

$x_t$ : 時刻tの入力 (batch_size, n_features)

$W_x$ : 入力に対する重み (n_features, n_nodes)

$h_{t-1}$ : 時刻t-1の状態（前の時刻から伝わる順伝播） (batch_size, n_nodes)

$W_h$ : 状態に対する重み。 (n_nodes, n_nodes)

$b$ : バイアス項 (1,)

初期状態 $h_0$ は全て0とすることが多いですが、任意の値を与えることも可能です。

上記の処理を系列数n_sequences回繰り返すことになります。RNN全体への入力 $x$ は(batch_size, n_sequences, n_features)のような配列で渡されることになり、そこから各時刻の配列を取り出していきます。

分類問題であれば、それぞれの時刻の$h$に対して全結合層とソフトマックス関数（またはシグモイド関数）を使用します。出力は最後の$h$だけを使用する場合と、全ての$h$を使う場合があります。

In [1]:
import numpy as np
class SimpleRNN:
    """
    シンプルなRNN
    
    Parameters
    ----------
    n_nodes : int
      ノード数
    n_features : int
      特徴量数
    initializer : インスタンス
        初期化方法のインスタンス
    optimizer : インスタンス
        最適化手法のインスタンス
    """
    def __init__(self, n_nodes, n_features, initializer, optimizer):
        self.optimizer = optimizer #最適化手法
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.Wx = initializer.W(n_features, n_nodes)
        self.Wh = initializer.W(n_nodes, n_nodes)
        self.B = initializer(1,)

        self.h = None
        
        
    def forward(self, X):
        """
        フォワードプロバケーション
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            入力
            
        Returns
        ----------
        self.h : 次の形のndarray, shape (batch_size, n_nodes)
            出力
        """
        #サイズの取得
        batch_size = X.shape[0] #バッチサイズ
        n_sequences = X.shape[1] #シーケンス数
        
        #hの初期値
        self.h = np.zeros([batch_size, n_nodes])

        #hを算出
        for i in range(n_sequences):
            X_t = np.dot(X[:, i, :], self.Wx) + np.dot(self.h, self.Wh) + self.B
            self.h = np.tanh(X_t)
        
        return self.h

## 【問題2】小さな配列でのフォワードプロパゲーションの実験

小さな配列でフォワードプロパゲーションを考えてみます。

入力$x$、初期状態$h$、重み$w\_x$と$w\_h$、バイアス$b$を次のようにします。

ここで配列xの軸はバッチサイズ、系列数、特徴量数の順番です。

In [2]:
x = np.array([[[1, 2], [2, 3], [3, 4]]])/100
w_x = np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
w_h = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
batch_size = x.shape[0] # 1
n_sequences = x.shape[1] # 3
n_features = x.shape[2] # 2
n_nodes = w_x.shape[1] # 4
h = np.zeros((batch_size, n_nodes))
b = np.array([1])

フォワードプロパゲーションの出力が次のようになることを作成したコードで確認してください。

In [3]:
#重みとバイアスは初期設定書き換え
class SimpleRNN:
    """
    シンプルなRNN
    
    Parameters
    ----------
    n_nodes1 : int
      前の層のノード数
    n_nodes2 : int
      後の層のノード数
    initializer : インスタンス
        初期化方法のインスタンス
    optimizer : インスタンス
        最適化手法のインスタンス
    """
    def __init__(self, n_nodes, n_features, initializer, optimizer):
        self.optimizer = optimizer #最適化手法
        
        # 初期化
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        #self.Wx = initializer.W(n_features, n_nodes)
        #self.Wh = initializer.W(n_nodes, n_nodes)
        #self.B = initializer(1,)
        
        #重みとバイアスの初期設定値
        self.Wx= np.array([[1, 3, 5, 7], [3, 5, 7, 8]])/100
        self.Wh = np.array([[1, 3, 5, 7], [2, 4, 6, 8], [3, 5, 7, 8], [4, 6, 8, 10]])/100
        self.B = np.array([1])
        
        self.h = None
        
        
    def forward(self, X):
        """
        フォワードプロバケーション
        Parameters
        ----------
        X : 次の形のndarray, shape (batch_size, n_sequences, n_features)
            入力
            
        Returns
        ----------
        self.h : 次の形のndarray, shape (batch_size, n_nodes)
            出力
        """
        #サイズの取得
        batch_size = X.shape[0] #バッチサイズ
        n_sequences = X.shape[1] #シーケンス数
        
        #hの初期値
        self.h = np.zeros([batch_size, n_nodes])

        #hを算出
        for i in range(n_sequences):
            X_t = np.dot(X[:, i, :], self.Wx) + np.dot(self.h, self.Wh) + self.B
            self.h = np.tanh(X_t)
        
        return self.h

In [4]:
#フォワードの実行
SimpleRNN(n_nodes, _, _, _).forward(x)

array([[0.79494228, 0.81839002, 0.83939649, 0.85584174]])

## 【問題3】（アドバンス課題）バックプロパゲーションの実装

バックプロパゲーションを実装します。

RNNの内部は全結合層を組み合わせた形になっているので、更新式は全結合層などと同様です。

$$W_x^{\prime} = W_x - \alpha E(\frac{\partial L}{\partial W_x})$$

$$W_h^{\prime} = W_h - \alpha E(\frac{\partial L}{\partial W_h})$$

$$b^{\prime} = b - \alpha E(\frac{\partial L}{\partial b})$$

$\alpha$ : 学習率

$\frac{\partial L}{\partial W_x}$ : $W_x$ に関する損失 $L$ の勾配
 
$\frac{\partial L}{\partial W_h}$ : $W_h$ に関する損失 $L$ の勾配
 
$\frac{\partial L}{\partial b}$ : $b$ に関する損失 $L$ の勾配
 
$E()$ : ミニバッチ方向にベクトルの平均を計算

勾配を求めるためのバックプロパゲーションの数式が以下です。

$\frac{\partial h_t}{\partial a_t} = \frac{\partial L}{\partial h_t} \times(1-tanh^2(a_t))$

$\frac{\partial L}{\partial b} = \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial W_x} = x_{t}^{T}\cdot \frac{\partial h_t}{\partial a_t}$

$\frac{\partial L}{\partial W_h} = h_{t-1}^{T}\cdot \frac{\partial h_t}{\partial a_t}$

＊$\frac{\partial L}{\partial h_t}$ は前の時刻からの状態の誤差と出力の誤差の合計です。hは順伝播時に出力と次の層に伝わる状態双方に使われているからです。

前の時刻や層に流す誤差の数式は以下です。

$\frac{\partial L}{\partial h_{t-1}} = \frac{\partial h_t}{\partial a_t}\cdot W_{h}^{T}$

$\frac{\partial L}{\partial x_{t}} = \frac{\partial h_t}{\partial a_t}\cdot W_{x}^{T}$

## 【問題2】（アドバンス課題）データセットでの学習・推定

これまで使ってきたニューラルネットワークにSimpleRNNを組み込み学習させ、動くことを確認してください。

[IMDB Review Dataset | Kaggle](https://www.kaggle.com/utathya/imdb-review-dataset)

映画レビューデータセットを使用します。ベクトル化を行い、作成したRNNに入力してください。